In [1]:
from keras_utilities import *
from keras_utilities.models.vgg16bn import Vgg16BN
from keras_utilities.models.vgg16 import Vgg16
from keras.layers import Dense, Merge
import os
from numpy import array
from IPython.display import FileLink
from numpy.random import permutation
from keras.preprocessing import image
from keras.applications import ResNet50
from keras.models import Sequential, Model
from keras.layers import AveragePooling2D, Input, Dense, Flatten
from keras import backend as K
from keras.optimizers import Adam
from keras_utilities import Vgg16, save_array, load_array

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 75.0% of memory, cuDNN 5103)
Using Theano backend.


In [2]:
prod = True

path = 'data/'
    
batch_size = 64

model_dir = "models/"
results_dir = "results/"

In [155]:
gen = image.ImageDataGenerator(
    rotation_range=20,
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     samplewise_std_normalization=True,
    zca_whitening=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    channel_shift_range=0.2,
    fill_mode="nearest"
)

batches = get_batches(path+"train", batch_size=128, shuffle=False)
val_batches = get_batches(path+"valid", batch_size=128, shuffle=False)

Found 20441 images belonging to 10 classes.
Found 1983 images belonging to 10 classes.


## ResNet

In [137]:
def finetune_resnet(nb_classes, lr=.001):
    rn = ResNet50()
    
    x = rn.layers[-4].output 
    mod = AveragePooling2D(name='avg_pool_ft', pool_size=(7,7))(x)
    mod = Flatten(name='flatten_ft')(mod)
    mod = Dense(nb_classes, name='dense_ft', activation='softmax')(mod)
    
    newmod = Model(input=rn.input, output=mod)
    
    for i in newmod.layers:
        if i.name in ['dense_ft']:
            i.trainable = True
        else:
            i.trainable = False
    
    opt = Adam(lr=lr)
    newmod.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
            
    return newmod
    

In [138]:
res_ft = finetune_resnet(10)

In [139]:
for i in res_ft.layers:
    print (i.name, i.trainable)

('input_33', False)
('zeropadding2d_13', False)
('conv1', False)
('bn_conv1', False)
('activation_589', False)
('maxpooling2d_13', False)
('res2a_branch2a', False)
('bn2a_branch2a', False)
('activation_590', False)
('res2a_branch2b', False)
('bn2a_branch2b', False)
('activation_591', False)
('res2a_branch2c', False)
('res2a_branch1', False)
('bn2a_branch2c', False)
('bn2a_branch1', False)
('merge_193', False)
('activation_592', False)
('res2b_branch2a', False)
('bn2b_branch2a', False)
('activation_593', False)
('res2b_branch2b', False)
('bn2b_branch2b', False)
('activation_594', False)
('res2b_branch2c', False)
('bn2b_branch2c', False)
('merge_194', False)
('activation_595', False)
('res2c_branch2a', False)
('bn2c_branch2a', False)
('activation_596', False)
('res2c_branch2b', False)
('bn2c_branch2b', False)
('activation_597', False)
('res2c_branch2c', False)
('bn2c_branch2c', False)
('merge_195', False)
('activation_598', False)
('res3a_branch2a', False)
('bn3a_branch2a', False)
('acti

In [118]:
res_ft.fit_generator(batches, 
                     batches.nb_sample, 
                     validation_data=val_batches, 
                     nb_val_samples = val_batches.nb_sample, 
                     nb_epoch=1)

Epoch 1/1
20441/20441 [==============================] - 282s - loss: 1.0949 - acc: 0.7232 - val_loss: 1.4893 - val_acc: 0.4665


In [57]:
res_ft.save_weights(model_dir+"resnet_1epoch.h5")

In [63]:
res_ft.fit_generator(batches, 
                     batches.nb_sample, 
                     validation_data=val_batches, 
                     nb_val_samples = val_batches.nb_sample, 
                     nb_epoch=1)

Epoch 1/1
20441/20441 [==============================] - 273s - loss: 0.3029 - acc: 0.9444 - val_loss: 1.3098 - val_acc: 0.5774


In [64]:
res_ft.save_weights(model_dir+"resnet_2epochs.h5")

In [65]:
res_ft.fit_generator(batches, 
                     batches.nb_sample, 
                     validation_data=val_batches, 
                     nb_val_samples = val_batches.nb_sample, 
                     nb_epoch=1)

Epoch 1/1
20441/20441 [==============================] - 274s - loss: 0.1928 - acc: 0.9669 - val_loss: 1.5236 - val_acc: 0.5204


In [80]:
res_ft

In [64]:
res_ft.save_weights(model_dir+"resnet_3epochs.h5")

In [45]:
def create_prediction_file(filename, preds=None, model=None):
    if not preds:
        num_test_files = len(os.listdir("data/test/unknown"))
        test_gen = get_batches("data/test", batch_size=128, shuffle=False)
        preds = model.predict_generator(test_gen, val_samples=num_test_files)
    elif not model and not preds:
        raise ValueError("Must pass either preds or a model.")
        
    files = os.listdir("data/test/unknown/")
        
    pred_str = map(lambda x: ",".join(["%.10f" % i for i in x]), preds.tolist())
    rows = [",".join([a,b]) + "\n" for a,b in zip(files, pred_str)]
    with open(filename, "a") as f:
        f.write("img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9\n")
        for row in rows:
            f.write(row)

In [58]:
num_test_files = len(os.listdir("data/test/unknown"))

In [14]:
test_gen = get_batches("data/test", shuffle=False, class_mode=None)
# preds = res_ft.predict_generator(test_gen, val_samples=num_test_files)

Found 79726 images belonging to 1 classes.


In [60]:
files = os.listdir("data/test/unknown/")

In [46]:
files[:10]

['img_81601.jpg',
 'img_14887.jpg',
 'img_62885.jpg',
 'img_45125.jpg',
 'img_22633.jpg',
 'img_45660.jpg',
 'img_88962.jpg',
 'img_92987.jpg',
 'img_26932.jpg',
 'img_57972.jpg']

In [61]:
filename = "results/resnet_3layer_ft.csv"
pred_str = map(lambda x: ",".join(["%.10f" % i for i in x]), preds.tolist())
rows = [",".join([a,b]) + "\n" for a,b in zip(files, pred_str)]
with open(filename, "a") as f:
    f.write("img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9\n")
    for row in rows:
        f.write(row)

In [62]:
FileLink("results/resnet_3layer_ft.csv")

/home/ubuntu/statefarm-kaggle/results/resnet_3layer_ft.csv

In [42]:
# as a comparison, let's see what the vgg model spits out for val loss
vgg = Vgg16()
vgg.ft(10)
vgg.fit(batches, val_batches)

Epoch 1/1
17456/17456 [==============================] - 550s - loss: 1.8595 - acc: 0.4785 - val_loss: 1.5956 - val_acc: 0.5076


In [ ]:
filename = "results/vanilla_vgg.csv"

preds_vgg = vgg.model.predict_generator(get_batches("data/test", batch_size=64, shuffle=False), 
                                        val_samples=len(os.listdir("data/test/unknown")))

pred_str = map(lambda x: ",".join(["%.10f" % i for i in x]), preds_vgg.tolist())
rows = [",".join([a,b]) + "\n" for a,b in zip(files, pred_str)]
with open(filename, "a") as f:
    f.write("img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9\n")
    for row in rows:
        f.write(row)

In [9]:
res_ft = finetune_resnet(10)

In [10]:
res_ft.load_weights(model_dir+"resnet_2epochs.h5")

In [68]:
preds = res_ft.predict_generator(test_gen, val_samples=test_gen.nb_sample)

In [ ]:
res_ft_trunc.fit_generator(batches, 
                           batches.nb_sample, 
                           validation_data=val_batches, 
                           nb_val_samples = val_batches.nb_sample, 
                           nb_epoch=1)

In [64]:
res_ft.save_weights(model_dir+"resnet_2epochs.h5")

In [ ]:
tuned_layer_inputs = res_ft_trunc.predict_generator(test_gen, val_samples=test_gen.nb_sample)

In [156]:
tuned_layer_inputs = intermediate_layer_output.predict_generator(batches, 
                                                                 batches.nb_sample)
save_array(path+"tuned_layer_inputs.bcolz", tuned_layer_inputs)
# tuned_layer_inputs = load_array(path+"tuned_layer_inputs.bcolz")

In [157]:
tuned_layer_val_inputs = intermediate_layer_output.predict_generator(val_batches, 
                                                                     val_batches.nb_sample)
save_array(path+"tuned_layer_val_inputs.bcolz", tuned_layer_val_inputs)
# tuned_layer_val_inputs = load_array(path+"tuned_layer_val_inputs.bcolz")

In [163]:
res_ft_trunc = ResNet50()

intermediate_layer_output = Model(input=res_ft_trunc.input, 
                                  output=res_ft_trunc.layers[-4].output)

intermediate_layer_output.compile(optimizer=Adam(), loss='categorical_crossentropy')

In [164]:
inputs = Input(shape=intermediate_layer_output.output_shape[1:])
mod = AveragePooling2D((7,7))(inputs)
mod = Flatten()(mod)
mod = Dense(10, name='wtf', activation='softmax', trainable=True)(mod)

In [165]:
mdl = Model(input=inputs, output=mod)

In [166]:
# for layer in mdl.layers:
#     if layer.name == 'dense_ft':
#         layer.trainable = True
#     else:
#         layer.trainable = False

In [167]:
mdl.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

mdl.fit(tuned_layer_inputs, onehot(batches.classes), nb_epoch=1, 
        validation_data=(tuned_layer_val_inputs, onehot(val_batches.classes)))



Train on 20441 samples, validate on 1983 samples
Epoch 1/1
20441/20441 [==============================] - 3s - loss: 0.7531 - acc: 0.7889 - val_loss: 1.7397 - val_acc: 0.4826
